In [1]:
import os
os.environ["OPENAI_API_KEY"] = "..."
os.environ["HUGGINGFACE_API_KEY"] = "..."

### Impact of Embeddings on Quality with Sub Question Query

In this tutorial, we load longer text (Alice in Wonderland) and utilize Llama-Index Sub Question Query to evlauate a complex questions on the book.

In addition, we will combine our choice of embedding with our query engine selection and use TruLens to select the most performant combination.

In [2]:
# NOTE: This is ONLY necessary in jupyter notebook.
# Details: Jupyter runs an event-loop behind the scenes. 
#          This results in nested event-loops when we start an event-loop to make async queries.
#          This is normally not allowed, we use nest_asyncio to allow it for convenience.  
import nest_asyncio
nest_asyncio.apply()

In [3]:
# Import main tools for building app
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.query_engine import SubQuestionQueryEngine

# load data
alice = SimpleDirectoryReader(input_dir="./data/alice").load_data()

In [8]:
# Imports main tools for eval
from trulens_eval import TruLlama, Feedback, Tru, feedback
tru = Tru()

#hugs = feedback.Huggingface()
openai = feedback.OpenAI()

# Question/answer relevance between overall question and answer.
model_agreement = Feedback(openai.model_agreement).on_input_output()

✅ In model_agreement, input prompt will be set to *.__record__.main_input or `Select.RecordInput` .
✅ In model_agreement, input response will be set to *.__record__.main_output or `Select.RecordOutput` .


In [7]:
# iterate through embeddings and chunk sizes, evaluating each response's agreement with chatgpt using TruLens
embeddings = ['text-embedding-ada-001','text-embedding-ada-002']
query_engine_types = ['VectorStoreIndex','SubQuestionQueryEngine']

service_context=512

for embedding in(embeddings):
    for query_engine_type in query_engine_types:

            # build index and query engine
            index = VectorStoreIndex.from_documents(alice)

            # create embedding-based query engine from index
            query_engine = index.as_query_engine(embed_model=embedding)

            if query_engine_type == 'SubQuestionQueryEngine':
                service_context = ServiceContext.from_defaults(chunk_size=512)
                # setup base query engine as tool
                query_engine_tools = [
                    QueryEngineTool(
                        query_engine=query_engine,
                        metadata=ToolMetadata(name='Alice in Wonderland', description='THE MILLENNIUM FULCRUM EDITION 3.0')
                    )
                ]
                query_engine = SubQuestionQueryEngine.from_defaults(query_engine_tools=query_engine_tools, service_context=service_context)
            else:
                pass         

            tc = TruLlama(app_id = f'{query_engine_type}_{embedding}', app = query_engine, feedbacks = [model_agreement])

            response = tc.query("Describe Alice's growth from meeting the White Rabbit to challenging the Queen of Hearts?")
            response = tc.query("Relate aspects of enchantment to the nostalgia that Alice experiences in Wonderland. Why is Alice both fascinated and frustrated by her encounters below-ground?")
            response = tc.query("Describe the White Rabbit's function in Alice.")
            response = tc.query("Describe some of the ways that Carroll achieves humor at Alice's expense.")
            response = tc.query("Compare the Duchess' lullaby to the 'You Are Old, Father William' verse")
            response = tc.query("Compare the sentiment of the Mouse's long tale, the Mock Turtle's story and the Lobster-Quadrille.")
            response = tc.query("Summarize the role of the mad hatter in Alice's journey")
            response = tc.query("How does the Mad Hatter influence the arc of the story throughout?")


✅ app VectorStoreIndex_text-embedding-ada-001 -> default.sqlite
✅ feedback def. feedback_definition_hash_9a0525b72342bf7c105c7f0b4260682c -> default.sqlite
✅ record record_hash_5753f20d341c6258d991ce9418a4a1bf from VectorStoreIndex_text-embedding-ada-001 -> default.sqlite
✅ record record_hash_7b318228e20b8d443a381bb576f5ec9b from VectorStoreIndex_text-embedding-ada-001 -> default.sqlite
DEBUG
 
You will continually start seeing responses to the prompt:

Compare the sentiment of the Mouse's long tale, the Mock Turtle's story and the Lobster-Quadrille.

The right answer is:


The sentiment of the Mouse's long tale is whimsical and humorous, while the Mock Turtle's story is sad and melancholy. The Lobster-Quadrille is lighthearted and cheerful.

Answer only with an integer from 1 to 10 based on how close the responses are to the right answer.

MODEL ANSWER
As a fact bot, I can provide information about the content of the Mouse's long tale, the Mock Turtle's story, and the Lobster-Quadrill

✅ record record_hash_49663d6690f011de4c2c305a692b0686 from SubQuestionQueryEngine_text-embedding-ada-002 -> default.sqlite
DEBUG
 
You will continually start seeing responses to the prompt:

How does the Mad Hatter influence the arc of the story throughout?

The right answer is:


The Mad Hatter influences the arc of the story throughout by providing Alice with advice and information about the strange world she has found herself in. He tells her about the Queen of Hearts' concert and how he and the March Hare quarreled before the March Hare went mad. He also explains why there are so many tea-things put out in the area. He then suggests that Alice tell a story, which leads to the Dormouse telling a story about three little sisters. Finally, the Mad Hatter provides Alice with directions to the March Hare's home and tells her that both he and the March Hare are mad. This helps Alice to understand the strange world she has found herself in and guides her on her journey.

Answer only with 

In [11]:
tru.run_dashboard()

Starting dashboard ...


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.15.216:8502 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>